In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['text.usetex'] = False
mpl.rcParams['font.size'] = 14
import xarray as xr
import pandas as pd
import scipy.io as sio
import sys
import os
import glob
import copy
import pathlib as pth
import importlib
import math
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.spice.splopter as spl
import flopter.spice.tdata as td
import flopter.core.ivdata as iv
import flopter.core.fitters as fts
import flopter.core.fitdata as fd
import flopter.core.lputils as lpu
import flopter.core.constants as c
import flopter.spice.inputparser as inp
import flopter.spice.normalise as nrm
import flopter.spice.xrfuncs as xrf
import flopter.spice.decomposition as dc

In [5]:
spice_dir = pth.Path('/home/jleland/data/external_big/spice/')
datasets_dir = 'sheath_exp_datasets/15V_cap'
datasets_path = spice_dir / datasets_dir
os.chdir(spice_dir)

In [4]:
bbm_ds = xr.load_dataset(f'{datasets_dir}/se_bbm_ivs.nc').isel(run=0)
fwp0_ds = xr.load_dataset(f'{datasets_dir}/se_fwp_0_ivs.nc').isel(run=0)
fwp2_ds = xr.load_dataset(f'{datasets_dir}/se_fwp_2_ivs.nc').isel(run=0)
fflwp_ds = xr.load_dataset(f'{datasets_dir}/se_fflwp_ivs.nc').isel(run=0)
hg_fflwp_ds = xr.load_dataset(f'{datasets_dir}/se_hg_fflwp_ivs.nc').isel(run=0)
hg_ds = xr.load_dataset(f'{datasets_dir}/se_hg_ivs.nc').isel(run=0)

combined_ds = xr.load_dataset(f'{datasets_dir}/se_combined.nc')

In [11]:
skippable_runs = set([
    'marconi/spice2/sheath_exp_hg/flat_flush_gapfill/alpha_yz_-6.0', # accidental duplicate
    'marconi/spice2/sheath_exp_hg/angled_recessed_as/alpha_yz_-2.0',
    'marconi/spice2/sheath_exp_hg/angled_recessed_as/alpha_yz_-3.0',
    'marconi/spice2/sheath_exp_hg_fflwp/angled_recessed_as/alpha_yz_-2.0',
    'marconi/spice2/sheath_exp_hg_fflwp/angled_recessed_as/alpha_yz_-3.0',
    'marconi/spice2/sheath_exp_hg_fflwp/angled_recessed_as/alpha_yz_-4.0',
    'marconi/spice2/sheath_exp_hg_fflwp/angled_recessed_as/alpha_yz_-5.0',
    'marconi/spice2/sheath_exp_hg_fflwp/angled_recessed_as/alpha_yz_-6.0',
    'marconi/spice2/sheath_exp_hg_fflwp/flat_flush_as/alpha_yz_-2.0',
    'marconi/spice2/sheath_exp_fflwp/angled_recessed_as/alpha_yz_-2.0',
    'marconi/spice2/sheath_exp_fwp/flat_flush_wp-2_as_1_/alpha_yz_-2.0',
    'marconi/spice2/bergmann_bm/flat_flush_lowas/alpha_yz_-1.5',
]) 
skippable_scans = set()
single_sims = set()

In [12]:
non_standard_variables = {'t', 'ProbePot', 'QnPot', 'npartproc', 'Nz', 'Nzmax', 'Ny', 'count', 'Npc', 'snumber', 'nproc'}
desired_variables = (td.DEFAULT_REDUCED_DATASET | non_standard_variables) - {td.OBJECTSCURRENTFLUXE, td.OBJECTSCURRENTFLUXI}

In [13]:

scans_searchstr = [
    '*/*/sheath_exp_hg/angled_recessed_as',
    '*/*/sheath_exp_fflwp/angled_recessed_as',
]
disallowed_angles = []

scans, all_run_dirs = xrf.get_run_dirs(scans_searchstr, skippable_runs=skippable_runs, 
                                       disallowed_angles=disallowed_angles)

[0]: marconi/spice2/sheath_exp_fflwp/angled_recessed_as
	[0,0]: angled_recessed_as/alpha_yz_-10.0
	[0,1]: angled_recessed_as/alpha_yz_-11.0
	[0,2]: angled_recessed_as/alpha_yz_-12.0
	[0,3]: angled_recessed_as/alpha_yz_-15.0
	[0,4]: angled_recessed_as/alpha_yz_-20.0
	[0,5]: angled_recessed_as/alpha_yz_-3.0
	[0,6]: angled_recessed_as/alpha_yz_-30.0
	[0,7]: angled_recessed_as/alpha_yz_-4.0
	[0,8]: angled_recessed_as/alpha_yz_-5.0
	[0,9]: angled_recessed_as/alpha_yz_-6.0
	[0,10]: angled_recessed_as/alpha_yz_-7.0
	[0,11]: angled_recessed_as/alpha_yz_-8.0
	[0,12]: angled_recessed_as/alpha_yz_-9.0
[1]: marconi/spice2/sheath_exp_hg/angled_recessed_as
	[1,0]: angled_recessed_as/alpha_yz_-10.0
	[1,1]: angled_recessed_as/alpha_yz_-12.0
	[1,2]: angled_recessed_as/alpha_yz_-20.0
	[1,3]: angled_recessed_as/alpha_yz_-30.0
	[1,4]: angled_recessed_as/alpha_yz_-5.0
	[1,5]: angled_recessed_as/alpha_yz_-6.0
	[1,6]: angled_recessed_as/alpha_yz_-8.0


In [14]:
spl_hg_path = spice_dir / all_run_dirs[scans[1]][0]
spl_hg_backup_path = list(spl_hg_path.glob('backup_*'))[-1]

spl_wg_path = spice_dir / all_run_dirs[scans[0]][0]
spl_wg_backup_path = list(spl_wg_path.glob('backup_*'))[-1]

print(f"halfgrid ex. - {all_run_dirs[scans[1]][0]}")
print(f"wholegrid ex. - {all_run_dirs[scans[0]][0]}")

halfgrid ex. - marconi/spice2/sheath_exp_hg/angled_recessed_as/alpha_yz_-10.0
wholegrid ex. - marconi/spice2/sheath_exp_fflwp/angled_recessed_as/alpha_yz_-10.0


# Preamble of necessary stuff

This bit creates all teh necessary variables for plotting to commence, including loading the dataset, creating a denormaliser, extracting relevant probe dimensions from an input file and defining a few functions for ease of manipulation. 

In [15]:
splopter_hg =  spl.Splopter(spl_hg_backup_path, reduce=desired_variables, ignore_tzero_fl=True, version=2.14,
                            store_dataframe_fl=True)
splopter_wg =  spl.Splopter(spl_wg_backup_path, reduce=desired_variables, ignore_tzero_fl=True, version=2.14,
                            store_dataframe_fl=True)

parser_hg = inp.InputParser(input_filename=spl_hg_backup_path / 'input.inp')
denormaliser_hg = nrm.Denormaliser(dt=splopter_hg.tdata.dt, input_parser=parser_hg) #, dimensions=3)

parser_wg = inp.InputParser(input_filename=spl_wg_backup_path / 'input.inp')
denormaliser_wg = nrm.Denormaliser(dt=splopter_wg.tdata.dt, input_parser=parser_wg)


Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_hg/angled_recessed_as/alpha_yz_-10.0/backup_20210111-0904) doesn't seem to be valid.
Continuing anyway.


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/scipy/io/matlab/mio.py:218: MatReadWarning: Duplicate variable name "npartproc" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_fflwp/angled_recessed_as/alpha_yz_-10.0/backup_20210211-0426) doesn't seem to be valid.
Continuing anyway.
dx = 1.0, dy = 0.5, dz = 0.5
dx = 1.0, dy = 1.0, dz = 1.0


In [10]:
# Quick look at sheath edge potential

fig, ax = plt.subplots()
ax.plot(splopter_hg.tdata.diagnostics['QnPot'])

In [11]:
probe_desig = {
    'angled_flush': {'theta_p':10.0, 'recession':0.0, 'gap': 1.0e-3},
    'angled_recessed': {'theta_p':10.0, 'recession':1.0e-3, 'gap': 1.0e-3},
    'flat_flush': {'theta_p':0.0, 'recession':0.0, 'gap': 1.0e-3},
    'flat_recessed': {'theta_p':0.0, 'recession':1.0e-3, 'gap': 1.0e-3},
    'flat_flush_gapless': {'theta_p':0.0, 'recession': 0.0, 'gap': 0.0}
}

tableau_palette = ['#377eb8', '#ff7f00', '#4daf4a', 
                   '#f781bf', '#a65628', '#984ea3', 
                   '#999999', '#e41a1c', '#dede00']

cb_palette = ['#4477AA', '#66CCEE', '#228833', '#CCBB44', 
              '#EE6677', '#AA3377', '#BBBBBB']

contrast_palette = ['#FFFFFF', '#DDAA33', '#BB5566', '#004488', '#000000']

vibrant_palette = ['#0077BB', '#33BBEE', '#009988', '#EE7733', 
                   '#CC3311', '#EE3377', '#BBBBBB']

muted_palette = ['#332288', '#88CCEE', '#44AA99', '#117733', '#999933', 
                 '#DDCC77', '#CC6677', '#882255', '#AA4499', '#DDDDDD']

paledark_palette = [
    ['#BBCCEE', '#CCEEFF', '#CCDDAA', '#EEEEBB', '#FFCCCC', '#DDDDDD'],
    ['#222255', '#225555', '#225522', '#666633', '#663333', '#555555'],
]

probe_colour = {probe_label: cb_palette[i] for i, probe_label in enumerate(probe_desig.keys())}
probe_colour

{'angled_flush': '#4477AA',
 'angled_recessed': '#66CCEE',
 'flat_flush': '#228833',
 'flat_recessed': '#CCBB44',
 'flat_flush_gapless': '#EE6677'}

In [16]:
dV = parser_wg.getfloat('mks', 'mks_te') #* c.BOLTZMANN / c.ELEM_CHARGE
T_e = parser_wg.getfloat('mks', 'mks_te')
n_0 = parser_wg.getfloat('mks', 'mks_n0')

L = parser_wg.getfloat('rectangle2', 'yhigh') - parser_wg.getfloat('rectangle2', 'ylow')
g = parser_wg.getfloat('rectangle2', 'ylow') - parser_wg.getfloat('rectangle0', 'yhigh') 
lambda_D = lpu.debye_length(T_e, n_0)
max_theta_p = np.radians(10.0)

print(f"T_e = {T_e} eV \n"
      f"n_0 = {n_0} m^-3 \n"
      f"L = {L} L_d \n"
      f"g = {g} L_d \n"
      f"lambda_D = {lambda_D} \n"
      f"theta_p = {max_theta_p} \n")

T_e = 5.0 eV 
n_0 = 1e+18 m^-3 
L = 300.0 L_d 
g = 60.0 L_d 
lambda_D = 1.6622799720325184e-05 
theta_p = 0.17453292519943295 



In [17]:
probe_desig = {
    'angled_flush': {'theta_p':10.0, 'recession':0.0, 'gap': 1.0e-3},
    'angled_recessed': {'theta_p':10.0, 'recession':1.0e-3, 'gap': 1.0e-3},
    'flat_flush': {'theta_p':0.0, 'recession':0.0, 'gap': 1.0e-3},
    'flat_recessed': {'theta_p':0.0, 'recession':1.0e-3, 'gap': 1.0e-3},
    'flat_flush_gapless': {'theta_p':0.0, 'recession': 0.0, 'gap': 0.0}
}

tableau_palette = ['#377eb8', '#ff7f00', '#4daf4a', 
                   '#f781bf', '#a65628', '#984ea3', 
                   '#999999', '#e41a1c', '#dede00']

cb_palette = ['#4477AA', '#66CCEE', '#228833', '#CCBB44', 
              '#EE6677', '#AA3377', '#BBBBBB']

contrast_palette = ['#FFFFFF', '#DDAA33', '#BB5566', '#004488', '#000000']

vibrant_palette = ['#0077BB', '#33BBEE', '#009988', '#EE7733', 
                   '#CC3311', '#EE3377', '#BBBBBB']

muted_palette = ['#332288', '#88CCEE', '#44AA99', '#117733', '#999933', 
                 '#DDCC77', '#CC6677', '#882255', '#AA4499', '#DDDDDD']

paledark_palette = [
    ['#BBCCEE', '#CCEEFF', '#CCDDAA', '#EEEEBB', '#FFCCCC', '#DDDDDD'],
    ['#222255', '#225555', '#225522', '#666633', '#663333', '#555555'],
]

probe_colour = {probe_label: cb_palette[i] for i, probe_label in enumerate(probe_desig.keys())}
probe_colour

{'angled_flush': '#4477AA',
 'angled_recessed': '#66CCEE',
 'flat_flush': '#228833',
 'flat_recessed': '#CCBB44',
 'flat_flush_gapless': '#EE6677'}

In [18]:
dummy_theta = np.linspace(0.00001, 45.0, 5000)
flush_calced_a = lpu.calc_sheath_expansion_param(
    T_e,
    n_0,
    L * lambda_D,
    g * lambda_D, 
    np.radians(dummy_theta),
    c_1=0.5,
    c_2=0.6,
)
probe_names = {'flat_flush': 'Flush-mounted', 
               'angled_recessed': 'Angled-tip Recessed'}

In [20]:
# Wholegrid floating sheath exp params, probe comparison

sel_args = {'theta':slice(2,30)}

fig, ax = plt.subplots(figsize=[8,6])

markers = ['o', '^', 's']
colours = (cb_palette[0], cb_palette[4])
mfcs = ['none', None]

plot_ds = fflwp_ds

for i, probe in enumerate(probe_names.keys()):

    probe_ds = plot_ds.sel(probe=probe, **sel_args)

    ax.errorbar(probe_ds.theta, probe_ds.ion_a, 
                yerr=probe_ds.ion_d_a, color=colours[i], mfc='none',
                label=f'{probe_names[probe]}', marker=markers[i], ls='none')
#     ax.errorbar(probe_ds.theta, probe_ds.mf_iv_a, 
#                 yerr=probe_ds.mf_iv_d_a, color=colours[i],
#                 label=f'{probes[probe]}', marker=markers[i], ls='none')
    
calced_a = lpu.calc_sheath_expansion_param(
    T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
    c_1=0.9, c_2=0.6,
)
ax.errorbar(dummy_theta, calced_a, label=r'Predicted - Murphy-Sugrue', fmt='-', 
            color=cb_palette[3], linewidth=1.0)

calced_a = lpu.calc_sheath_expansion_param(
    T_e, n_0, L * lambda_D, 0, np.radians(dummy_theta), 
    c_1=0.5, c_2=0.6,
)
ax.errorbar(dummy_theta, calced_a, label=r'Predicted - Bergmann', fmt='--', 
            color=cb_palette[2], linewidth=1.0)
    

ax.set_ylim(5e-3, 6e-2)
ax.set_xlim(-0.5, 30.5)
ax.set_ylabel(r'$a$ [unitless]')
ax.set_xlabel(r'$\theta$ [$^{\circ}$]')

ax.legend()
fig.tight_layout()